<a href="https://colab.research.google.com/github/amirkasaei/Disease-Prediction/blob/main/Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries**

In [193]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Softmax, BatchNormalization, Dropout
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ModelCheckpoint
from google.colab import drive
from keras.utils.np_utils import to_categorical
import json

In [ ]:
drive.mount('/content/drive')
path = "/content/drive/MyDrive/AI Projects/Disease-Prediction/"

# **Preprocessing Data**

In [113]:
dis_symp = pd.read_csv(path + 'Data/dataset.csv')
dis_symp = dis_symp[:100]
cols = dis_symp.columns[1:]
symptoms = np.array(dis_symp[cols].values.tolist())
print(symptoms.shape)
symptoms = symptoms[symptoms != 'nan']
print(symptoms.shape)

(100, 17)
(481,)


In [115]:
print(np.unique(symptoms).shape)

(44,)


In [215]:
training = pd.read_csv(path + 'Data/Training.csv')
testing = pd.read_csv(path + 'Data/Testing.csv')

cols= training.columns
cols= cols[:-1]
x = np.array(training[cols].values.tolist())

y = np.array(training['prognosis'].values.tolist())
le = LabelEncoder()
le.fit(y)
output_classes = le.classes_
y = le.transform(y)
y = y.reshape((-1, 1))
y_OH = to_categorical(y, num_classes=len(output_classes))

x_train, x_val, y_train, y_val = train_test_split(x, y_OH, test_size=0.20, random_state=42)

In [217]:
print('classes: ', output_classes)

import pickle
file_ = open(path + 'Model/output_classes.pkl', "wb")
pickle.dump(output_classes, file_)
file_.close()

classes:  ['(vertigo) Paroymsal  Positional Vertigo' 'AIDS' 'Acne'
 'Alcoholic hepatitis' 'Allergy' 'Arthritis' 'Bronchial Asthma'
 'Cervical spondylosis' 'Chicken pox' 'Chronic cholestasis' 'Common Cold'
 'Dengue' 'Diabetes ' 'Dimorphic hemmorhoids(piles)' 'Drug Reaction'
 'Fungal infection' 'GERD' 'Gastroenteritis' 'Heart attack' 'Hepatitis B'
 'Hepatitis C' 'Hepatitis D' 'Hepatitis E' 'Hypertension '
 'Hyperthyroidism' 'Hypoglycemia' 'Hypothyroidism' 'Impetigo' 'Jaundice'
 'Malaria' 'Migraine' 'Osteoarthristis' 'Paralysis (brain hemorrhage)'
 'Peptic ulcer diseae' 'Pneumonia' 'Psoriasis' 'Tuberculosis' 'Typhoid'
 'Urinary tract infection' 'Varicose veins' 'hepatitis A']


In [156]:
print(y_OH[:2])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [182]:
print('x_train.shape = ', x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_test.shape = ', x_val.shape)
print('y_test.shape = ', y_val.shape)

x_train.shape =  (3936, 132)
y_train.shape =  (3936, 41)
x_test.shape =  (984, 132)
y_test.shape =  (984, 41)


# **Neural Network**

In [190]:
input_size = x.shape[1]

model = Sequential()
model.add(Dense(90, input_shape=(input_size,), activation='relu'))
model.add(Dense(70, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(41, activation='softmax'))

checkpoint_filepath = path + '/Model/weights.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True, verbose=0)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x, y_OH, epochs=10, batch_size=256, validation_data=(x_val, y_val), verbose=1, callbacks=[checkpoint])

In [235]:
# test
test = x[1200:1201]
print(test)
print(np.argmax(y_OH[1200]))
print(np.argmax(model.predict(test, verbose=0)))

[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
38
38


# **Load model and use it**

In [233]:
path = "/content/drive/MyDrive/AI Projects/Disease-Prediction/"
model_loc = path + '/Model/weights.hdf5'
classList_loc = path + 'Model/output_classes.pkl'

# read output_classes using pickle
file_ = open(classList_loc, "rb")
output_classes = pickle.load(file_)
file_.close()

def predict(features):
  model = load_model(model_loc)
  result = model.predict(np.array([features]), verbose=0)
  class_num = np.argmax(result)

  return output_classes[class_num]

In [234]:
print(x[0])
print(output_classes[15])
print(predict(x[0]))

[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Fungal infection
Fungal infection
